In [4]:
import os
import uuid
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse

SAVE_FOLDER = "raw_docs_scraped"

TARGET_URLS = [
    "https://www.bluedart.com/fsc",
    "https://www.travarc.in/couriers/bluedart/charges",
    "https://www.delhivery.com/services/express",
    "https://www.delhivery.com/fuelsurcharge",
    "https://www.dtdc.in/fuel-surcharge-factor.asp",
    "https://www.travarc.in/couriers/vrl/charges",
    "https://www.tyntec.com/blogs/whatsapp-message-templates-logistics/",
    "https://www.gorgias.com/blog/useful-email-templates-for-customer-service",
    "https://www.smscountry.com/blog/whatsapp-message-templates-customer-support"
]

HEADERS = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/120.0.0.0 Safari/537.36"
    )
}

def sanitize_filename(url):
    parsed = urlparse(url)
    safe = parsed.netloc + parsed.path.replace("/", "_").replace("?", "_")
    return safe if safe else str(uuid.uuid4())

def scrape(url):
    print(f"\n🔍 Scraping: {url}")

    try:
        response = requests.get(url, headers=HEADERS, timeout=20)
        response.raise_for_status()
    except Exception as e:
        print(f"ERROR fetching {url}: {e}")
        return

    soup = BeautifulSoup(response.text, "html.parser")

    for tag in soup(["script", "style", "nav", "header", "footer", "img"]):
        tag.decompose()

    text = soup.get_text(separator="\n")

    lines = [line.strip() for line in text.splitlines() if line.strip()]
    cleaned = "\n".join(lines)

    os.makedirs(SAVE_FOLDER, exist_ok=True)

    filename = sanitize_filename(url) + ".txt"
    path = os.path.join(SAVE_FOLDER, filename)

    with open(path, "w", encoding="utf-8") as f:
        f.write(cleaned)

    print(f"Saved → {path}")


if __name__ == "__main__":
    print("\nAUTO SCRAPER STARTED...")
    print(f"Saving files in: {SAVE_FOLDER}")

    for url in TARGET_URLS:
        scrape(url)

    print("\nDONE! All working pages scraped successfully.")



AUTO SCRAPER STARTED...
Saving files in: raw_docs_scraped

🔍 Scraping: https://www.bluedart.com/fsc
ERROR fetching https://www.bluedart.com/fsc: 404 Client Error: Not Found for url: https://www.bluedart.com/fsc

🔍 Scraping: https://www.travarc.in/couriers/bluedart/charges
ERROR fetching https://www.travarc.in/couriers/bluedart/charges: 404 Client Error: Not Found for url: https://www.travarc.in/couriers/bluedart/charges

🔍 Scraping: https://www.delhivery.com/services/express
Saved → raw_docs_scraped\www.delhivery.com_services_express.txt

🔍 Scraping: https://www.delhivery.com/fuelsurcharge
Saved → raw_docs_scraped\www.delhivery.com_fuelsurcharge.txt

🔍 Scraping: https://www.dtdc.in/fuel-surcharge-factor.asp
Saved → raw_docs_scraped\www.dtdc.in_fuel-surcharge-factor.asp.txt

🔍 Scraping: https://www.travarc.in/couriers/vrl/charges
ERROR fetching https://www.travarc.in/couriers/vrl/charges: 404 Client Error: Not Found for url: https://www.travarc.in/couriers/vrl/charges

🔍 Scraping: http

In [6]:
import os
import uuid
import pdfplumber

RAW_PDF_FOLDER = "raw_pdfs"
OUTPUT_FOLDER = "raw_docs_combined"

os.makedirs(OUTPUT_FOLDER, exist_ok=True)

for file in os.listdir(RAW_PDF_FOLDER):
    if file.endswith(".pdf"):
        path = os.path.join(RAW_PDF_FOLDER, file)
        print("Extracting:", file)

        text = ""
        with pdfplumber.open(path) as pdf:
            for page in pdf.pages:
                text += page.extract_text() + "\n"

        clean_name = file.replace(".pdf", "") + ".txt"
        output_path = os.path.join(OUTPUT_FOLDER, clean_name)

        with open(output_path, "w", encoding="utf-8") as f:
            f.write(text)

        print("Saved:", output_path)


Extracting: Indian Carriage of Goods by Sea Act, 1925.pdf
Saved: raw_docs_combined\Indian Carriage of Goods by Sea Act, 1925.txt
Extracting: logistics-services-contract-template-sample.pdf
Saved: raw_docs_combined\logistics-services-contract-template-sample.txt
Extracting: motor_transport_workers_act_1961.pdf
Saved: raw_docs_combined\motor_transport_workers_act_1961.txt
Extracting: the_arbitration_and_conciliation_act,_1996.pdf
Saved: raw_docs_combined\the_arbitration_and_conciliation_act,_1996.txt


In [7]:
import os
import shutil

COMBINED = "raw_docs_combined"
SCRAPED = "raw_docs_scraped"

os.makedirs(COMBINED, exist_ok=True)

for folder in [SCRAPED]:
    for file in os.listdir(folder):
        source = os.path.join(folder, file)
        dest = os.path.join(COMBINED, file)
        shutil.copy(source, dest)

print("✔ All text files merged into raw_docs_combined/")


✔ All text files merged into raw_docs_combined/


In [8]:
import os
import re

INPUT_FOLDER = "raw_docs_combined"
OUTPUT_FOLDER = "cleaned_docs"

os.makedirs(OUTPUT_FOLDER, exist_ok=True)

def clean_text(text):
    text = re.sub(r"\s+", " ", text)            # Remove multiple spaces
    text = re.sub(r"Page \d+ of \d+", "", text) # Remove page numbers
    text = re.sub(r"©.*?\n", "", text)          # Remove copyright lines
    text = text.replace("•", "- ")              # Normalize bullets
    return text.strip()

for file in os.listdir(INPUT_FOLDER):
    path = os.path.join(INPUT_FOLDER, file)
    with open(path, "r", encoding="utf-8") as f:
        text = f.read()

    cleaned = clean_text(text)

    with open(os.path.join(OUTPUT_FOLDER, file), "w", encoding="utf-8") as f:
        f.write(cleaned)

print("✔ Cleaned text stored in cleaned_docs/")


✔ Cleaned text stored in cleaned_docs/


In [9]:
import os
import re
import uuid

INPUT_FOLDER = "cleaned_docs"
CLAUSE_FOLDER = "clauses"

os.makedirs(CLAUSE_FOLDER, exist_ok=True)

def split_into_clauses(text):
    # Main patterns of clause boundaries:
    patterns = [
        r"\.\s",              # sentences ending in period
        r";\s",               # semicolons
        r"\n-\s",             # bullet points
        r"\n\d+\.\s",         # numbered clauses
        r"\n\([a-z]\)\s",     # sub-clauses (a), (b)
    ]

    regex = "|".join(patterns)
    parts = re.split(regex, text)

    # Filter long clauses
    clauses = [p.strip() for p in parts if len(p.strip()) > 20]

    return clauses

for file in os.listdir(INPUT_FOLDER):
    path = os.path.join(INPUT_FOLDER, file)
    with open(path, "r", encoding="utf-8") as f:
        text = f.read()

    clauses = split_into_clauses(text)

    for clause in clauses:
        file_id = str(uuid.uuid4()) + ".txt"
        with open(os.path.join(CLAUSE_FOLDER, file_id), "w", encoding="utf-8") as f:
            f.write(clause)

print("✔ All documents split into clauses → clauses/ folder")


✔ All documents split into clauses → clauses/ folder


In [13]:
import os
import uuid
import json
import requests
import textstat

CLAUSE_FOLDER = "clauses"
OUTPUT_FOLDER = "metadata"
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

OLLAMA_URL = "http://localhost:11434/api/generate"
MODEL = "llama3.1"  # your local model


def classify_clause(clause_text):
    prompt = f"""
You are a legal-logistics contract classifier.

Classify the following clause into:

1. category 
   (SLA, Pricing, Penalty, Liability, Force Majeure, Termination, 
    Confidentiality, Dispute Resolution, Definitions, Exceptions)

2. risk_type 
   (delay, damage, weather, compliance, payment, general)

3. conditions 
   (comma-separated conditions like monsoon, peak hours, fragile, temperature-controlled)

4. jurisdiction 
   (India, Global, or Unknown)

5. summary 
   (1-line description)

Return the output as STRICT JSON only.

Clause:
\"{clause_text}\"
"""

    payload = {
        "model": MODEL,
        "prompt": prompt,
        "stream": False
    }

    try:
        response = requests.post(OLLAMA_URL, json=payload)
        result = response.json()["response"]

        # Extract JSON
        json_start = result.find("{")
        json_end = result.rfind("}") + 1
        metadata = json.loads(result[json_start:json_end])

        return metadata

    except Exception as e:
        print("Error processing clause:", e)
        return None


for file in os.listdir(CLAUSE_FOLDER):
    clause_path = os.path.join(CLAUSE_FOLDER, file)

    with open(clause_path, "r", encoding="utf-8") as f:
        clause_text = f.read().strip()

    clause_id = str(uuid.uuid4())

    metadata = classify_clause(clause_text)
    if metadata is None:
        continue

    metadata["clause_id"] = clause_id
    metadata["text"] = clause_text
    metadata["flesch_score"] = textstat.flesch_reading_ease(clause_text)
    metadata["industry"] = "Logistics"

    # Save JSON
    output_path = os.path.join(OUTPUT_FOLDER, clause_id + ".json")
    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(metadata, f, indent=4)

    print(f"Tagged: {clause_id}")


Tagged: feffb0a2-e005-46b8-908e-aa4c5895aa1c
Error processing clause: Expecting value: line 1 column 1 (char 0)
Tagged: 10e1f791-f43b-478f-83c3-ebe7cbacf379
Tagged: ea3d4485-bef3-46ca-8913-8284d4601e97
Tagged: 5266dacf-dd94-4f76-bf5e-0ab018577ea1
Tagged: f3389dc0-92ac-490a-8ff5-3190042666a5
Tagged: bf9f373b-53d1-4f02-8f50-3af0ffe04d3e
Tagged: 8063df79-4ea1-4646-b182-5fa9372a1795
Tagged: 628e4f73-124f-4a62-a579-282b9e7ef1dc
Tagged: 55ebb344-c967-4b3c-abc4-6537461999ee
Tagged: 61e49acf-d8f7-4f22-a158-65285dfd3078
Error processing clause: Expecting value: line 1 column 1 (char 0)
Tagged: 7859109a-1544-42a8-a6b4-9533b1ebb601
Tagged: ea447a45-c59c-47d1-b780-a5d457f48114
Tagged: a019a950-7de5-42b8-bfc2-8a8e5ce78389
Tagged: bcb76006-4f4f-463c-ae18-6a9da3b8ea53
Tagged: dff6f70b-aa08-4bf4-9374-a7c6413d63a2
Tagged: a962649c-320b-4fab-97c8-a929ac1a110c
Error processing clause: Expecting value: line 1 column 1 (char 0)
Tagged: 8e082511-9715-4238-aa87-54c74119e36e
Tagged: 506e3118-a341-4f62-841b-94

KeyboardInterrupt: 

In [4]:
import requests
print(requests.post(
    "http://localhost:11434/api/embed",
    json={"model": "mxbai-embed-large", "input": "hello world"}
).json())


{'model': 'mxbai-embed-large', 'embeddings': [[0.022850003, 0.03220857, 0.01650996, -0.0404585, -0.021860443, -0.007932052, 0.029880203, 0.037131645, 0.04611216, 0.024687622, 0.006918501, 0.019998133, -0.012432475, 0.0074579227, -0.046731215, 0.02023465, -0.025149453, -0.030046446, -0.06062744, 0.03276451, -0.023770848, 0.012671855, -0.08374644, -0.03310265, -0.025545636, 0.041583564, 0.027566042, -0.00032868667, 0.041532602, 0.043370675, -0.011174011, 0.0031959773, 0.001222711, -0.050825235, -0.011090056, -0.015285167, 0.044858236, -0.024148898, -0.024531659, -0.041625306, 0.029241692, 0.00313352, 0.030501647, -0.0397974, -0.06958308, -0.013245187, 0.026166862, 0.0048879297, 0.046337064, -0.004029227, 0.0018304469, 0.040606245, 0.009097129, -0.0012182103, -0.011813936, 0.013545584, -0.060890988, 0.010417787, -0.031790137, 0.010683298, 0.024557035, 0.052827705, 0.02045037, -0.072703585, 0.012181791, 0.04279134, 0.0055577313, -0.0052666664, 0.03612027, 0.01514817, -0.03719298, 0.0535082

In [5]:
import os
import json
import requests

CLAUSE_FOLDER = "clauses"
METADATA_FOLDER = "metadata"
OUTPUT_FILE = "embeddings.jsonl"

OLLAMA_URL = "http://localhost:11434/api/embed"
EMBED_MODEL = "mxbai-embed-large"

with open(OUTPUT_FILE, "w", encoding="utf-8") as out:
    for meta_file in os.listdir(METADATA_FOLDER):
        path = os.path.join(METADATA_FOLDER, meta_file)

        with open(path, "r", encoding="utf-8") as f:
            metadata = json.load(f)

        clause_id = metadata["clause_id"]
        clause_text = metadata["text"]

        payload = {"model": EMBED_MODEL, "input": clause_text}
        response = requests.post(OLLAMA_URL, json=payload)
        embedding = response.json()["embeddings"][0]

        record = {
            "id": clause_id,
            "values": embedding,
            "metadata": metadata
        }

        out.write(json.dumps(record) + "\n")

        print("Embedded:", clause_id)


Embedded: 0064392d-4db3-458f-a025-30ce88787190
Embedded: 00b597ad-1c71-462c-98c8-fc2f4bf1ef03
Embedded: 00d21622-c56c-475f-9e5e-173cf0b1adce
Embedded: 00dba929-3179-4cdc-95d8-d2079560f3b4
Embedded: 00e59fca-0926-435e-91d3-b303767986cd
Embedded: 00ff312d-5971-4881-beae-83f2c180e189
Embedded: 015fd992-a685-4fde-a4c2-1dd2f0f5959f
Embedded: 01ac78ca-9e24-402a-89c6-2eaca5554ecf
Embedded: 01f08da3-6e20-4049-b851-e14dcc949e5b
Embedded: 022e0785-411e-4896-8250-d454a13a13d2
Embedded: 024e725a-d76e-429a-82b4-eaeff1dc9c27
Embedded: 025207cd-e065-4569-a49c-2a91ed0964c6
Embedded: 0258e221-2fd4-4df8-915b-b810a859c8b8
Embedded: 02b93d17-6537-496f-bc89-5ef17f34ad8c
Embedded: 02de4eb2-5839-4221-b02d-ca5bcfc90e53
Embedded: 02f96039-1eaa-45d1-8dbe-d404e3155a27
Embedded: 0385bdfd-3ddc-496f-b8d8-948a19654963
Embedded: 03ae59e5-ab94-469b-8f7e-9e6d8d2ae2a6
Embedded: 03c2ce72-e02d-4fbd-8cb8-624cd9b5e3fc
Embedded: 03f9fad0-0bad-4939-8ee3-1f8d083dcc22
Embedded: 0443a5bf-df43-4c92-9d28-5c93100c1ecd
Embedded: 045

In [7]:
from pymilvus import (
    connections, FieldSchema, CollectionSchema,
    DataType, Collection
)

connections.connect("default", host="127.0.0.1", port="19540")

embedding_dim = 768  # dimension of mxbai embeddings

fields = [
    FieldSchema(
        name="clause_id",
        dtype=DataType.VARCHAR,
        is_primary=True,
        max_length=64
    ),
    FieldSchema(
        name="embedding",
        dtype=DataType.FLOAT_VECTOR,
        dim=embedding_dim
    ),
    FieldSchema(
        name="category",
        dtype=DataType.VARCHAR,
        max_length=50
    ),
    FieldSchema(
        name="risk_type",
        dtype=DataType.VARCHAR,
        max_length=50
    ),
    FieldSchema(
        name="jurisdiction",
        dtype=DataType.VARCHAR,
        max_length=50
    ),
    FieldSchema(
        name="summary",
        dtype=DataType.VARCHAR,
        max_length=512
    ),
]

schema = CollectionSchema(
    fields,
    description="Logistics contract clauses collection"
)

collection = Collection("logistics_clauses", schema)

# Create index
index_params = {
    "metric_type": "COSINE",
    "index_type": "IVF_FLAT",
    "params": {"nlist": 1024}
}

collection.create_index("embedding", index_params)

print("✅ Collection created: logistics_clauses")


✅ Collection created: logistics_clauses


In [12]:
from pymilvus import connections, utility
connections.connect("default", host="127.0.0.1", port="19540")
utility.drop_collection("logistics_clauses")


In [13]:
from pymilvus import (
    connections, FieldSchema, CollectionSchema, DataType, Collection
)

connections.connect("default", host="127.0.0.1", port="19540")

fields = [
    FieldSchema(name="id", dtype=DataType.VARCHAR, max_length=64, is_primary=True),
    FieldSchema(name="embedding", dtype=DataType.FLOAT_VECTOR, dim=1024),
    FieldSchema(name="category", dtype=DataType.VARCHAR, max_length=200),
    FieldSchema(name="risk_type", dtype=DataType.VARCHAR, max_length=200),
    FieldSchema(name="jurisdiction", dtype=DataType.VARCHAR, max_length=200),
    FieldSchema(name="summary", dtype=DataType.VARCHAR, max_length=500),
]

schema = CollectionSchema(fields, description="Clause embeddings")
collection = Collection("logistics_clauses", schema)


In [14]:
import json
from pymilvus import connections, Collection

connections.connect("default", host="127.0.0.1", port="19540")
collection = Collection("logistics_clauses")

ids = []
vectors = []
categories = []
risks = []
jurisdictions = []
summaries = []

def safe(value, default="unknown"):
    if value is None:
        return default
    if value == "":
        return default
    return str(value)

with open("embeddings.jsonl", "r", encoding="utf-8") as f:
    for line in f:
        rec = json.loads(line)
        meta = rec["metadata"]

        ids.append(rec["id"])
        vectors.append(rec["values"])

        categories.append(safe(meta.get("category")))
        risks.append(safe(meta.get("risk_type")))
        jurisdictions.append(safe(meta.get("jurisdiction")))
        summaries.append(safe(meta.get("summary"), default="no-summary"))

collection.insert([
    ids, vectors, categories, risks, jurisdictions, summaries
])

collection.flush()

print("✅ All embeddings inserted into Milvus (NULL-safe)!")


✅ All embeddings inserted into Milvus (NULL-safe)!


In [17]:
from pymilvus import Collection

collection = Collection("logistics_clauses")

index_params = {
    "index_type": "IVF_FLAT",
    "metric_type": "COSINE",
    "params": {"nlist": 1024}
}

collection.create_index(
    field_name="embedding",
    index_params=index_params
)

print("Index created!")


Index created!


In [18]:
import requests
from pymilvus import connections, Collection

connections.connect("default", host="127.0.0.1", port="19540")
collection = Collection("logistics_clauses")

query = "penalty for monsoon delays"

payload = {"model": "mxbai-embed-large", "input": query}
embedding = requests.post(
    "http://localhost:11434/api/embed", json=payload
).json()["embeddings"][0]

collection.load()

results = collection.search(
    data=[embedding],
    anns_field="embedding",
    param={"metric_type": "COSINE", "params": {"nprobe": 10}},
    limit=5,
    output_fields=["category", "summary", "jurisdiction"]
)

print(results)


data: [[{'id': '60b56cf4-874b-4b23-a9c9-8a277c039dd8', 'distance': 0.6817986369132996, 'entity': {'category': 'SLA', 'summary': 'Deliveries may be delayed in flood-affected areas due to weather conditions.', 'jurisdiction': 'Global'}}, {'id': '7a5787f5-9ebd-4577-98c2-5b715021406d', 'distance': 0.6438724994659424, 'entity': {'category': 'Liability', 'summary': 'Penalty for disobedience or contravention of Act provisions', 'jurisdiction': 'India'}}, {'id': 'ed0b7278-866a-429d-8fb4-f89ffab8a09b', 'distance': 0.6294867992401123, 'entity': {'category': 'Liability', 'summary': 'Accountability for monthly duration of delays', 'jurisdiction': 'Global'}}, {'id': '711a23bf-860e-4cee-b316-2fb7cd8a1588', 'distance': 0.6116024851799011, 'entity': {'category': 'Liability', 'summary': 'Penalty for obstructing an inspector during inspection, examination, or inquiry under the Act.', 'jurisdiction': 'India'}}, {'id': '0258e221-2fd4-4df8-915b-b810a859c8b8', 'distance': 0.5948134064674377, 'entity': {'cat

In [45]:
import re
import json
import requests
from pymilvus import connections, Collection


def llama_json_call(prompt: str):
    """
    Calls LLaMA through Ollama using streaming mode.
    Collects all chunks, reconstructs full text, then attempts JSON parse.
    """
    url = "http://localhost:11434/api/generate"

    resp = requests.post(
        url,
        json={"model": "llama3.1", "prompt": prompt, "stream": True},
        stream=True
    )

    full_text = ""

    for line in resp.iter_lines():
        if not line:
            continue
        try:
            data = json.loads(line.decode())
            if "response" in data:
                full_text += data["response"]
        except:
            continue

    full_text = full_text.strip()

    # Try JSON
    try:
        return json.loads(full_text)
    except:
        return full_text


def extract_dates(text):
    pattern = r"\b(\d{1,2}[/-]\d{1,2}[/-]\d{2,4}|\b\d{4}\b)\b"
    return re.findall(pattern, text)

def extract_money(text):
    pattern = r"\b(?:Rs\.?|INR|₹|\$)\s?\d+(?:,\d{3})*(?:\.\d+)?\b"
    return re.findall(pattern, text)

def extract_percentage(text):
    return re.findall(r"\b\d{1,3}%\b", text)

def extract_duration(text):
    return re.findall(r"\b(\d+\s?(days?|weeks?|months?|years?))\b", text)


JURISDICTION_KEYWORDS = {
    "india": "India",
    "global": "Global",
    "international": "Global",
    "uae": "UAE",
    "us": "USA",
    "usa": "USA",
    "europe": "Europe"
}

def extract_jurisdiction(text):
    text_lower = text.lower()
    for key, val in JURISDICTION_KEYWORDS.items():
        if key in text_lower:
            return val
    return "Unknown"



def classify_query_with_llama(query):
    prompt = f"""
Classify the user query into:
- intent
- category

User Query: "{query}"

Return JSON only:
{{
  "intent": "...",
  "category": "..."
}}
"""
    resp = llama_json_call(prompt)

    if isinstance(resp, dict):
        return resp

    try:
        return json.loads(resp)
    except:
        return {"intent": "unknown", "category": "unknown"}


def llama_entity_extract(query):
    prompt = f"""
Extract the following entities:

- event
- shipment_type
- damage_type
- delay_reason
- weather_condition
- party_names

Return JSON only.
Query: "{query}"
"""
    resp = llama_json_call(prompt)

    if isinstance(resp, dict):
        return resp

    try:
        return json.loads(resp)
    except:
        return {}



def interpret_query(query):

    dates = extract_dates(query)
    money = extract_money(query)
    perc = extract_percentage(query)
    duration = extract_duration(query)
    jurisdiction = extract_jurisdiction(query)

    cls = classify_query_with_llama(query)
    llm_entities = llama_entity_extract(query)

    return {
        "intent": cls["intent"],
        "category": cls["category"],
        "jurisdiction": jurisdiction,
        "entities": {
            "dates": dates,
            "amounts": money,
            "percentages": perc,
            "duration": duration,
            **llm_entities
        },
        "raw_query": query
    }



def embed(text):
    payload = {"model": "mxbai-embed-large", "input": text}
    resp = requests.post("http://localhost:11434/api/embed", json=payload).json()
    return resp["embeddings"][0]



connections.connect("default", host="127.0.0.1", port="19540")
collection = Collection("logistics_clauses")
collection.load()

def search_milvus(query_embedding, top_k=10):
    results = collection.search(
        data=[query_embedding],
        anns_field="embedding",
        param={"metric_type": "COSINE", "params": {"nprobe": 10}},
        limit=top_k,
        output_fields=["category", "summary", "jurisdiction"]
    )

    hits = []
    for hit in results[0]:
        hits.append({
            "id": hit.id,
            "distance": hit.distance,
            "category": hit.entity.get("category"),
            "summary": hit.entity.get("summary"),
            "jurisdiction": hit.entity.get("jurisdiction")
        })
    return hits



def rerank(query, hits):
    scored = []
    for h in hits:
        prompt = f"""
Rate how relevant this clause summary is to the query (0-1 scale).

Query: {query}
Clause Summary: {h['summary']}

Return only a NUMBER.
"""
        resp = llama_json_call(prompt)

        try:
            score = float(resp)
        except:
            score = 0.0

        h["rerank_score"] = score
        scored.append(h)

    scored.sort(key=lambda x: x["rerank_score"], reverse=True)
    return scored



def generate_answer(query, ner, top_clause):
    prompt = f"""
You are an AI Contract Analyst.

Your job is to generate a structured JSON output with the following format:

{{
  "answer": "Short human-friendly answer.",
  "clause_used": "The clause summary used to generate the answer.",
  "is_penalty_applicable": true/false,
  "reasoning": "Why this answer was generated.",
  "confidence": 0.0 to 1.0,
  "final_output": "Clean final sentence replying to the user."
}}

Instructions:
- Use ONLY information from the clause and NER.
- If weather-related delays, strikes, floods, or natural disasters appear → often no penalty.
- If delay_reason includes negligence, compliance issue, or avoidable event → penalty may apply.
- Confidence must be a number between 0 and 1.

User Query:
{query}

NER Extracted:
{json.dumps(ner, indent=2)}

Relevant Clause Summary:
{top_clause['summary']}

Generate STRICT JSON only.
"""

    resp = llama_json_call(prompt)

    # Ensure final output is JSON dict
    if isinstance(resp, dict):
        return resp

    try:
        return json.loads(resp)
    except:
        return {
            "answer": "Unable to parse model output.",
            "clause_used": top_clause['summary'],
            "is_penalty_applicable": False,
            "reasoning": "Model returned unstructured output.",
            "confidence": 0.0,
            "final_output": "System error occurred."
        }




class ContractEngine:

    def ask(self, query):

        ner = interpret_query(query)      # 1. NER
        vec = embed(query)                # 2. Embedding
        results = search_milvus(vec)      # 3. Vector search
        ranked = rerank(query, results)   # 4. Reranking
        answer = generate_answer(query, ner, ranked[0])  # 5. Final answer

        return {
            "answer": answer,
            "supporting_clause": ranked[0],
            "ner": ner
        }


# Create instance
contract_engine = ContractEngine()


In [85]:
response = contract_engine.ask(
    "What penalty applies if delivery is delayed 3 days due to heavy rain?"
)

print(response["answer"])
response["supporting_clause"]
response["ner"]


{'answer': 'No penalty likely, as delivery was delayed due to heavy rain.', 'clause_used': 'Deliveries may be delayed in flood-affected areas due to weather conditions.', 'is_penalty_applicable': False, 'reasoning': 'Weather-related delay; clause does not mention negligence or compliance issues.', 'confidence': 0.85, 'final_output': 'No penalty is likely, as the delivery was delayed due to heavy rain.'}


{'intent': 'unknown',
 'category': 'unknown',
 'jurisdiction': 'Unknown',
 'entities': {'dates': [],
  'amounts': [],
  'percentages': [],
  'duration': [('3 days', 'days')]},
 'raw_query': 'What penalty applies if delivery is delayed 3 days due to heavy rain?'}

In [86]:
import random
import math
import json

# =========================================================
# 1. SURCHARGE NAMES (7 surcharge factors)
# =========================================================

SURCHARGES = [
    "fuel_surcharge",               # FSC
    "peak_season_surcharge",        # PEAK
    "residential_fee",              # RES
    "dimensional_weight_fee",       # DIM
    "congestion_fee",               # CONG
    "risk_surcharge",               # RISK
    "carbon_emission_fee"           # CARBON
]

# Default bounds per surcharge
SURCHARGE_LIMITS = {
    "fuel_surcharge": (0.05, 0.25),              # 5% to 25%
    "peak_season_surcharge": (0.00, 0.20),       # 0% to 20%
    "residential_fee": (20, 200),                # ₹20 to ₹200
    "dimensional_weight_fee": (1.0, 1.5),        # 1x to 1.5x multiplier
    "congestion_fee": (0, 300),                  # ₹0 to ₹300
    "risk_surcharge": (0.00, 0.10),              # 0% to 10%
    "carbon_emission_fee": (0.005, 0.02)         # 0.5% to 2%
}


# =========================================================
# 2. MARKET SIGNALS (dynamic risk, fuel, season)
# =========================================================

def get_market_signal():
    """Simulates market-based adjustments."""
    fuel_index = random.uniform(0.8, 1.3)        # diesel price effect
    season_index = random.uniform(0.9, 1.4)      # peak season effect
    route_risk_index = random.uniform(0.7, 1.5)  # geopolitical/weather risk
    return fuel_index, season_index, route_risk_index


def adjust_expected_surcharges():
    fuel_idx, season_idx, risk_idx = get_market_signal()

    expected = {}
    for s in SURCHARGES:
        base_low, base_high = SURCHARGE_LIMITS[s]
        base_value = (base_low + base_high) / 2

        if "fuel" in s:
            expected[s] = base_value * fuel_idx
        elif "peak" in s:
            expected[s] = base_value * season_idx
        elif "risk" in s:
            expected[s] = base_value * risk_idx
        else:
            expected[s] = base_value

    return expected


# =========================================================
# 3. UTILITY FUNCTIONS
# =========================================================

def carrier_utility(s):
    """Carrier prefers higher surcharges (more revenue)."""
    α = {
        "fuel_surcharge": 8,
        "peak_season_surcharge": 4,
        "residential_fee": 2,
        "dimensional_weight_fee": 6,
        "congestion_fee": 3,
        "risk_surcharge": 5,
        "carbon_emission_fee": 1
    }

    utility = sum(α[k] * s[k] for k in SURCHARGES)
    return utility


def shipper_utility(s):
    """Shipper prefers lower surcharges (less cost)."""
    γ = {
        "fuel_surcharge": 10,
        "peak_season_surcharge": 6,
        "residential_fee": 3,
        "dimensional_weight_fee": 7,
        "congestion_fee": 4,
        "risk_surcharge": 5,
        "carbon_emission_fee": 2
    }

    utility = -sum(γ[k] * s[k] for k in SURCHARGES)
    return utility


# =========================================================
# 4. INITIAL OFFERS
# =========================================================

def generate_initial_offers(expected):
    """Carrier starts high, shipper starts low."""
    carrier_offer = {}
    shipper_offer = {}

    for s in SURCHARGES:
        low, high = SURCHARGE_LIMITS[s]
        carrier_offer[s] = min(high, expected[s] * 1.2)
        shipper_offer[s] = max(low, expected[s] * 0.6)

    return carrier_offer, shipper_offer


# =========================================================
# 5. NEGOTIATION ROUND (Rubinstein bargaining)
# =========================================================

def counter_offer(previous, direction="down", strength=0.1):
    """Generate a counter-offer in negotiation."""
    updated = {}

    for s in SURCHARGES:
        low, high = SURCHARGE_LIMITS[s]
        value = previous[s]

        if direction == "down":          # shipper -> reduce cost
            new_val = value - value * strength
        else:                            # carrier -> increase cost
            new_val = value + value * strength

        updated[s] = max(low, min(high, new_val))

    return updated


# =========================================================
# 6. NEGOTIATION LOOP
# =========================================================

def negotiate(max_rounds=12):
    expected = adjust_expected_surcharges()

    carrier_offer, shipper_offer = generate_initial_offers(expected)

    print("\n=== NEGOTIATION START ===")
    for round in range(1, max_rounds + 1):
        print(f"\n--- Round {round} ---")
        print("Carrier offer:", carrier_offer)
        print("Shipper offer:", shipper_offer)

        # Compute utilities
        Uc = carrier_utility(carrier_offer)
        Us = shipper_utility(shipper_offer)

        print(f"Carrier Utility: {Uc:.2f}")
        print(f"Shipper Utility: {Us:.2f}")

        # Check convergence
        diff = sum(abs(carrier_offer[s] - shipper_offer[s]) for s in SURCHARGES)

        if diff < 0.05:
            print("\nNegotiation converged!")
            return finalize_offer(carrier_offer, shipper_offer)

        # Carrier proposes next counter-offer
        carrier_offer = counter_offer(shipper_offer, direction="up", strength=0.08)

        # Shipper proposes next counter-offer
        shipper_offer = counter_offer(carrier_offer, direction="down", strength=0.10)

    print("\nNegotiation ended (max rounds reached).")
    return finalize_offer(carrier_offer, shipper_offer)


# =========================================================
# 7. FINAL AGREEMENT
# =========================================================

def finalize_offer(c_offer, s_offer):
    """Weighted average based on negotiation strength."""
    wc = 0.55   # carrier bargaining power
    ws = 0.45   # shipper bargaining power

    final = {}
    for s in SURCHARGES:
        final[s] = round((c_offer[s] * wc + s_offer[s] * ws), 4)

    print("\n=== FINAL AGREEMENT ===")
    print(json.dumps(final, indent=4))

    return final


# =========================================================
# RUN NEGOTIATION
# =========================================================

if __name__ == "__main__":
    final_contract_terms = negotiate()



=== NEGOTIATION START ===

--- Round 1 ---
Carrier offer: {'fuel_surcharge': 0.19304473325428953, 'peak_season_surcharge': 0.1212264373224418, 'residential_fee': 132.0, 'dimensional_weight_fee': 1.5, 'congestion_fee': 180.0, 'risk_surcharge': 0.07028475282604363, 'carbon_emission_fee': 0.015}
Shipper offer: {'fuel_surcharge': 0.09652236662714477, 'peak_season_surcharge': 0.0606132186612209, 'residential_fee': 66.0, 'dimensional_weight_fee': 1.0, 'congestion_fee': 90.0, 'risk_surcharge': 0.035142376413021814, 'carbon_emission_fee': 0.0075}
Carrier Utility: 815.40
Shipper Utility: -566.52

--- Round 2 ---
Carrier offer: {'fuel_surcharge': 0.10424415595731634, 'peak_season_surcharge': 0.06546227615411858, 'residential_fee': 71.28, 'dimensional_weight_fee': 1.08, 'congestion_fee': 97.2, 'risk_surcharge': 0.03795376652606356, 'carbon_emission_fee': 0.0081}
Shipper offer: {'fuel_surcharge': 0.0938197403615847, 'peak_season_surcharge': 0.05891604853870672, 'residential_fee': 64.152, 'dimensi

In [87]:
import requests
import random
import json
import math
import time


GLOBAL_FUEL_API = "https://api.globalfuelprices.com/india/diesel"
OPENWEATHER_API = "https://api.openweathermap.org/data/2.5/weather"
TOMTOM_API = "https://api.tomtom.com/traffic/services/4/flowSegmentData/relative/10/json"

OPENWEATHER_KEY = "31dd0e1363193b8fffec768ce53f182c"
TOMTOM_KEY = "1bITluIAYeilmu8OUxkYK26zOyk8YJhr"

SURCHARGES = [
    "fuel_surcharge",
    "peak_season_surcharge",
    "residential_fee",
    "dimensional_weight_fee",
    "congestion_fee",
    "risk_surcharge",
    "carbon_emission_fee"
]

SURCHARGE_LIMITS = {
    "fuel_surcharge": (0.05, 0.25),
    "peak_season_surcharge": (0.00, 0.20),
    "residential_fee": (20, 200),
    "dimensional_weight_fee": (1.0, 1.5),
    "congestion_fee": (0, 300),
    "risk_surcharge": (0.00, 0.10),
    "carbon_emission_fee": (0.005, 0.02)
}


def get_live_fuel_index():
    try:
        resp = requests.get(GLOBAL_FUEL_API, timeout=10).json()
        price = resp["price"]
        # Normalize diesel price (100 = baseline)
        return price / 100
    except Exception:
        print("⚠ Fuel API failed → Using fallback index")
        return random.uniform(0.9, 1.2)


def get_weather_risk(city):
    """Weather → increases risk surcharge."""
    try:
        resp = requests.get(
            OPENWEATHER_API,
            params={"q": city, "appid": OPENWEATHER_KEY},
            timeout=10
        ).json()

        weather = resp["weather"][0]["main"].lower()
        
        if any(w in weather for w in ["rain", "storm", "flood", "cyclone"]):
            return 1.35
        if "snow" in weather:
            return 1.28
        return 1.0
    except:
        print("⚠ Weather API failed → Using fallback risk")
        return 1.1


def get_traffic_congestion(lat, lon):
    """TomTom returns congestion level 0–100."""
    try:
        resp = requests.get(
            TOMTOM_API,
            params={
                "point": f"{lat},{lon}",
                "key": TOMTOM_KEY
            },
            timeout=10
        ).json()

        flow = resp["flowSegmentData"]
        congestion = flow["currentSpeed"] / flow["freeFlowSpeed"]
        congestion_index = 1 / congestion
        return min(max(congestion_index, 1.0), 1.5)
    except:
        print("⚠ Traffic API failed → Using fallback congestion index")
        return random.uniform(1.05, 1.25)


def adjust_expected_surcharges(city="Mumbai", lat=19.07, lon=72.87):
    fuel_idx = get_live_fuel_index()
    weather_idx = get_weather_risk(city)
    congestion_idx = get_traffic_congestion(lat, lon)

    expected = {}

    for s in SURCHARGES:
        low, high = SURCHARGE_LIMITS[s]
        mid = (low + high) / 2

        if s == "fuel_surcharge":
            expected[s] = mid * fuel_idx

        elif s == "peak_season_surcharge":
            season_multiplier = random.uniform(1.0, 1.25)
            expected[s] = mid * season_multiplier

        elif s == "congestion_fee":
            expected[s] = mid * congestion_idx

        elif s == "risk_surcharge":
            expected[s] = mid * weather_idx

        else:
            expected[s] = mid

    return expected

def carrier_utility(s):
    α = {
        "fuel_surcharge": 8,
        "peak_season_surcharge": 4,
        "residential_fee": 2,
        "dimensional_weight_fee": 6,
        "congestion_fee": 3,
        "risk_surcharge": 5,
        "carbon_emission_fee": 1
    }
    return sum(α[k] * s[k] for k in SURCHARGES)


def shipper_utility(s):
    γ = {
        "fuel_surcharge": 10,
        "peak_season_surcharge": 6,
        "residential_fee": 3,
        "dimensional_weight_fee": 7,
        "congestion_fee": 4,
        "risk_surcharge": 5,
        "carbon_emission_fee": 2
    }
    return -sum(γ[k] * s[k] for k in SURCHARGES)



def generate_initial_offers(expected):
    carrier_offer = {}
    shipper_offer = {}

    for s in SURCHARGES:
        low, high = SURCHARGE_LIMITS[s]

        carrier_offer[s] = min(high, expected[s] * 1.25)
        shipper_offer[s] = max(low, expected[s] * 0.55)

    return carrier_offer, shipper_offer


def counter_offer(prev, direction="down", strength=0.10):
    new_offer = {}

    for s in SURCHARGES:
        low, high = SURCHARGE_LIMITS[s]
        value = prev[s]

        if direction == "down":
            updated = value * (1 - strength)
        else:
            updated = value * (1 + strength)

        new_offer[s] = max(low, min(high, updated))

    return new_offer


def negotiate(city="Mumbai", lat=19.07, lon=72.87, max_rounds=10):

    expected = adjust_expected_surcharges(city, lat, lon)
    carrier_offer, shipper_offer = generate_initial_offers(expected)

    print("\n=== LIVE NEGOTIATION STARTED ===")

    for r in range(1, max_rounds+1):
        print(f"\nRound {r}")
        print("Carrier:", carrier_offer)
        print("Shipper:", shipper_offer)

        diff = sum(abs(carrier_offer[s] - shipper_offer[s]) for s in SURCHARGES)

        if diff < 0.05:
            print("\nConverged!")
            return finalize_offer(carrier_offer, shipper_offer)

        carrier_offer = counter_offer(shipper_offer, direction="up", strength=0.08)
        shipper_offer = counter_offer(carrier_offer, direction="down", strength=0.09)

    print("\n⚠ Max rounds reached → Forcing agreement")
    return finalize_offer(carrier_offer, shipper_offer)


def finalize_offer(c, s):
    wc, ws = 0.55, 0.45
    final = {}

    for sur in SURCHARGES:
        final[sur] = round(c[sur] * wc + s[sur] * ws, 4)

    print("\n=== FINAL AGREEMENT ===")
    print(json.dumps(final, indent=4))
    return final


if __name__ == "__main__":
    negotiate("Mumbai", 19.07, 72.87)


⚠ Fuel API failed → Using fallback index

=== LIVE NEGOTIATION STARTED ===

Round 1
Carrier: {'fuel_surcharge': 0.2142804631881277, 'peak_season_surcharge': 0.12520308623993942, 'residential_fee': 137.5, 'dimensional_weight_fee': 1.5, 'congestion_fee': 187.5, 'risk_surcharge': 0.0625, 'carbon_emission_fee': 0.015625}
Shipper: {'fuel_surcharge': 0.0942834038027762, 'peak_season_surcharge': 0.055089357945573354, 'residential_fee': 60.50000000000001, 'dimensional_weight_fee': 1.0, 'congestion_fee': 82.5, 'risk_surcharge': 0.027500000000000004, 'carbon_emission_fee': 0.006875000000000001}

Round 2
Carrier: {'fuel_surcharge': 0.1018260761069983, 'peak_season_surcharge': 0.05949650658121922, 'residential_fee': 65.34000000000002, 'dimensional_weight_fee': 1.08, 'congestion_fee': 89.10000000000001, 'risk_surcharge': 0.029700000000000004, 'carbon_emission_fee': 0.007425000000000001}
Shipper: {'fuel_surcharge': 0.09266172925736846, 'peak_season_surcharge': 0.054141820988909496, 'residential_fee'

In [89]:
# ============================================================
# FINAL INTEGRATION → Contract Generator
# ============================================================

def create_surcharge_clause(terms):
    return f"""
--- Negotiated Commercial Terms ---

Fuel Surcharge: {terms['fuel_surcharge']*100:.2f}%
Peak Season Surcharge: {terms['peak_season_surcharge']*100:.2f}%
Residential Fee: ₹{terms['residential_fee']:.2f}
Dimensional Weight Factor: {terms['dimensional_weight_fee']:.3f}x
Congestion Fee: ₹{terms['congestion_fee']:.2f}
Risk Surcharge: {terms['risk_surcharge']*100:.2f}%
Carbon Emission Fee: {terms['carbon_emission_fee']*100:.2f}%
"""


def generate_contract(query):
    print("\n\n=== GENERATING INTELLIGENT CONTRACT ===\n")

    # 1. NER
    ner = interpret_query(query)

    # 2. RAG retrieval
    vec = embed(query)
    base_results = search_milvus(vec)
    ranked = rerank(query, base_results)
    top_clause = ranked[0]

    # 3. Negotiation
    negotiated_terms = negotiate()

    # 4. Build Contract Output
    contract = f"""
===========================================
     AI-GENERATED INTELLIGENT CONTRACT
===========================================

Query Asked:
{query}

-------------------------------------------
1. NER Interpretation:
-------------------------------------------
{json.dumps(ner, indent=2)}

-------------------------------------------
2. Clause Used:
-------------------------------------------
Summary: {top_clause['summary']}
Jurisdiction: {top_clause['jurisdiction']}
Category: {top_clause['category']}

-------------------------------------------
3. Negotiated Commercial Terms:
-------------------------------------------
{create_surcharge_clause(negotiated_terms)}

-------------------------------------------
4. Full Answer to User:
-------------------------------------------
{generate_answer(query, ner, top_clause)['final_output']}

===========================================
End of Contract — Generated Automatically
===========================================
"""

    return contract


In [90]:
print(generate_contract(
    "What penalty applies if delivery is delayed 3 days due to heavy rain?"
))




=== GENERATING INTELLIGENT CONTRACT ===

⚠ Fuel API failed → Using fallback index

=== LIVE NEGOTIATION STARTED ===

Round 1
Carrier: {'fuel_surcharge': 0.21407733291559541, 'peak_season_surcharge': 0.14681685609338208, 'residential_fee': 137.5, 'dimensional_weight_fee': 1.5, 'congestion_fee': 187.5, 'risk_surcharge': 0.0625, 'carbon_emission_fee': 0.015625}
Shipper: {'fuel_surcharge': 0.09419402648286199, 'peak_season_surcharge': 0.06459941668108812, 'residential_fee': 60.50000000000001, 'dimensional_weight_fee': 1.0, 'congestion_fee': 82.5, 'risk_surcharge': 0.027500000000000004, 'carbon_emission_fee': 0.006875000000000001}

Round 2
Carrier: {'fuel_surcharge': 0.10172954860149096, 'peak_season_surcharge': 0.06976737001557518, 'residential_fee': 65.34000000000002, 'dimensional_weight_fee': 1.08, 'congestion_fee': 89.10000000000001, 'risk_surcharge': 0.029700000000000004, 'carbon_emission_fee': 0.007425000000000001}
Shipper: {'fuel_surcharge': 0.09257388922735678, 'peak_season_surcha

In [77]:
import datetime
import json

class ContractComposer:

    def __init__(self):
        self.template_header = """
=====================================================================
                 HYBRID MASTER SERVICE AGREEMENT (MSA)
                         + ANNUAL RATE CONTRACT (ARC)
=====================================================================
This Agreement (“Agreement”) is made on {date} between:

1. **Carrier / Logistics Provider:** {carrier_name}
2. **Shipper / Client:** {shipper_name}

Jurisdiction: {jurisdiction}

This Agreement supersedes all prior discussions and defines the terms,
conditions, obligations, and commercial framework under which the Carrier
shall provide logistics and transportation services to the Shipper.
=====================================================================
"""

        # Auto Chapters (each 2–4 pages)
        self.chapters = {
            "definitions": "## 1. DEFINITIONS\n" + ("Definition clauses...\n" * 40),
            "scope": "## 2. SCOPE OF WORK\n" + ("Detailed scope of logistics...\n" * 40),
            "sla": "## 3. SERVICE LEVEL AGREEMENTS\n" + ("SLA clauses...\n" * 60),
            "penalties": "## 4. PENALTIES & LIABILITIES\n" + ("Liability clauses...\n" * 60),
            "pricing": "## 5. PRICING & COMMERCIAL TERMS\n{pricing_table}\n",
            "adjustments": "## 6. FUEL & WEATHER ADJUSTMENTS\n" + ("Adjustment rules...\n" * 40),
            "dim": "## 7. DIMENSIONAL WEIGHT RULES\n" + ("DIM rules...\n" * 40),
            "surcharges": "## 8. SURCHARGES APPLICABLE\n{negotiated_surcharges}\n",
            "billing": "## 9. BILLING & SETTLEMENT\n" + ("Billing rules...\n" * 40),
            "termination": "## 10. TERMINATION\n" + ("Termination rules...\n" * 40),
            "confidentiality": "## 11. CONFIDENTIALITY\n" + ("Confidentiality clauses...\n" * 40),
            "compliance": "## 12. COMPLIANCE & INDEMNITY\n" + ("Compliance clauses...\n" * 40),
            "data": "## 13. DATA PROTECTION & API ACCESS\n" + ("API/data clauses...\n" * 40),
            "force": "## 14. FORCE MAJEURE\n" + ("Force majeure clauses...\n" * 40),
            "disputes": "## 15. DISPUTE RESOLUTION\n" + ("Dispute rules...\n" * 40),
        }

        # Annexures
        self.annexures = {
            "annex_a": "### ANNEXURE A — SERVICE LEVEL MATRIX\n" + ("SLA table...\n" * 40),
            "annex_b": "### ANNEXURE B — PRICING TABLE\n{pricing_table}\n",
            "annex_c": "### ANNEXURE C — SURCHARGE RULES\n{negotiated_surcharges}\n",
            "annex_d": "### ANNEXURE D — PACKAGING GUIDELINES\n" + ("Packaging rules...\n" * 40),
            "annex_e": "### ANNEXURE E — TRANSIT RISK RULES\n" + ("Transit risk...\n" * 40),
            "annex_f": "### ANNEXURE F — PENALTY REFERENCE INDEX\n" + ("Penalty details...\n" * 40),
            "annex_g": "### ANNEXURE G — IT/API INTEGRATION SPEC\n" + ("API integration details...\n" * 40),
            "annex_h": "### ANNEXURE H — CONTACT POINTS\n" + ("Contacts...\n" * 40),
        }

    def compose_contract(self, ner, top_clause, negotiated_terms):

        carrier_name = ner["entities"].get("party_names", ["Carrier"])[0] if ner["entities"].get("party_names") else "Carrier Company"
        shipper_name = ner["entities"].get("party_names", ["Shipper"])[-1] if ner["entities"].get("party_names") else "Shipper Company"

        jurisdiction = ner["jurisdiction"]

        # Convert negotiated terms to table
        pricing_table = "\n".join(
            [f"- **{k.replace('_',' ').title()}**: {v}" for k, v in negotiated_terms.items()]
        )

        # Also create surcharge block
        surcharge_block = "\n".join(
            [f"{k.title().replace('_',' ')}: {v}" for k, v in negotiated_terms.items()]
        )

        contract = self.template_header.format(
            date=datetime.date.today(),
            carrier_name=carrier_name,
            shipper_name=shipper_name,
            jurisdiction=jurisdiction
        )

        # Add chapters
        for key, text in self.chapters.items():
            contract += "\n\n" + text.format(
                pricing_table=pricing_table,
                negotiated_surcharges=surcharge_block
            )

        # Add Annexures (these alone create ~20 pages)
        contract += "\n\n================ ANNEXURES ================\n\n"

        for key, text in self.annexures.items():
            contract += "\n\n" + text.format(
                pricing_table=pricing_table,
                negotiated_surcharges=surcharge_block
            )

        return contract


In [83]:
composer = ContractComposer()

contract_text = composer.compose_contract(
    ner=response["ner"],
    top_clause=response["supporting_clause"],
    negotiated_terms=final_contract_terms
)

with open("INTELLIGENT_CONTRACT.txt", "w", encoding="utf-8") as f:
    f.write(contract_text)


In [84]:
final_contract_terms = negotiate("Mumbai", 19.07, 72.87)


⚠ Fuel API failed → Using fallback index

=== LIVE NEGOTIATION STARTED ===

Round 1
Carrier: {'fuel_surcharge': 0.17993461785112397, 'peak_season_surcharge': 0.1453088886806101, 'residential_fee': 137.5, 'dimensional_weight_fee': 1.5, 'congestion_fee': 187.5, 'risk_surcharge': 0.0625, 'carbon_emission_fee': 0.015625}
Shipper: {'fuel_surcharge': 0.07917123185449455, 'peak_season_surcharge': 0.06393591101946845, 'residential_fee': 60.50000000000001, 'dimensional_weight_fee': 1.0, 'congestion_fee': 82.5, 'risk_surcharge': 0.027500000000000004, 'carbon_emission_fee': 0.006875000000000001}

Round 2
Carrier: {'fuel_surcharge': 0.08550493040285412, 'peak_season_surcharge': 0.06905078390102594, 'residential_fee': 65.34000000000002, 'dimensional_weight_fee': 1.08, 'congestion_fee': 89.10000000000001, 'risk_surcharge': 0.029700000000000004, 'carbon_emission_fee': 0.007425000000000001}
Shipper: {'fuel_surcharge': 0.07780948666659725, 'peak_season_surcharge': 0.06283621334993361, 'residential_fee'